In [233]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split

In [277]:
filepath_dict={
    'train':'Train_and_test.txt'
}

In [278]:
df_list=[]
for source, filepath in filepath_dict.items():
  dfn=pd.read_csv(filepath, names=['label','sent'], sep='\t')
  df['source']=source
  df_list.append(df)
df


,label,sent,source
0,O,All,train
1,O,live,train
2,O,births,train
3,O,>,train
4,O,or,train
...,...,...,...
68170,T,globulin,train
68171,O,for,train
68172,D,recurrent,train
68173,D,spontaneous,train


In [236]:
df.dtypes

label     object
sent      object
source    object
dtype: object

In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62489 entries, 0 to 68174
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   62489 non-null  object
 1   sent    62489 non-null  object
 2   source  62489 non-null  object
dtypes: object(3)
memory usage: 1.9+ MB


In [238]:
df=pd.concat(df_list)

In [239]:
df_train=df[df['source']=='train']

In [240]:
sentences=df_train['sent'].values
sentences=sentences.astype(str)

In [241]:
import numpy as np
y=df_train['label'].values
y=y.astype(str)
sentences_train, sentences_test, y_train, y_test= train_test_split(
  sentences, y, test_size=0.25, random_state=1000  
)


In [242]:
tokenizer=Tokenizer(num_words=5000)

In [245]:
tokenizer.fit_on_texts(sentences_train)
X_train=tokenizer.texts_to_sequences(sentences_train)
X_test=tokenizer.texts_to_sequences(sentences_test)

In [246]:
vocab_size=len(tokenizer.word_index)+1
embedding_dim=100
maxlen=100
X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

In [247]:
cnn=Sequential()
cnn.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
cnn.add(layers.Conv1D(128,5, activation='relu'))
cnn.add(layers.GlobalMaxPool1D())
cnn.add(layers.Dense(10, activation='relu'))
cnn.add(layers.Dense(1, activation='sigmoid'))
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 100)          797700    
                                                                 
 conv1d_13 (Conv1D)          (None, 96, 128)           64128     
                                                                 
 global_max_pooling1d_12 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_21 (Dense)            (None, 10)                1290      
                                                                 
 dense_22 (Dense)            (None, 1)                 11        
                                                                 
Total params: 863,129
Trainable params: 863,129
Non-trainable params: 0
_______________________________________________

In [248]:
X_train

array([[2692,    0,    0, ...,    0,    0,    0],
       [ 664,    0,    0, ...,    0,    0,    0],
       [1095,    0,    0, ...,    0,    0,    0],
       ...,
       [   2,    0,    0, ...,    0,    0,    0],
       [   4,    0,    0, ...,    0,    0,    0],
       [3090,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [249]:
y_train

array(['O', 'O', 'O', ..., 'O', 'O', 'D'], dtype='<U7')

In [260]:
print(y_train.shape)


(46866, 1)


In [261]:
y_train = y_train.reshape(-1,1)


In [263]:
print(y_train.shape)


(46866, 1)


In [264]:
from sklearn import preprocessing
import numpy as np
enc = preprocessing.OrdinalEncoder(categories='auto')

enc.fit(y_train)
print( enc.transform(y_train) )

[[2.]
 [2.]
 [2.]
 ...
 [2.]
 [2.]
 [1.]]


In [265]:
y_train = enc.transform(y_train)

In [267]:
y_test


array([['O'],
       ['O'],
       ['D'],
       ...,
       ['O'],
       ['O'],
       ['O']], dtype='<U7')

In [268]:
print(y_test.shape)

(15623, 1)


In [269]:
en = preprocessing.OrdinalEncoder(categories='auto')

en.fit(y_test)
print( enc.transform(y_test) )

[[2.]
 [2.]
 [1.]
 ...
 [2.]
 [2.]
 [2.]]


In [270]:
y_test=enc.transform(y_test)

In [272]:
history=cnn.fit(X_train, y_train, epochs=10, verbose=False, validation_data=(X_test,y_test), batch_size=10)


In [273]:
loss, accuracy = cnn.evaluate(X_train, y_train, verbose=False)

In [274]:
print("Training Accuracy: {:.4f}".format(accuracy))

Training Accuracy: 0.0780


In [275]:
loss, accuracy = cnn.evaluate(X_test, y_test, verbose=False)

In [276]:
print("Testting Accuracy: {:.4f}".format(accuracy))

Testting Accuracy: 0.0776


In [307]:
y_pred=cnn.predict(X_test)

In [308]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         1.0       0.08      1.00      0.14      1212
         2.0       0.00      0.00      0.00     13525
         3.0       0.00      0.00      0.00       886

    accuracy                           0.08     15623
   macro avg       0.03      0.33      0.05     15623
weighted avg       0.01      0.08      0.01     15623



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
